In [30]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
import numpy as np
import pandas as pd
import torch

In [4]:
MODEL_NAME = "512-encoder-1000K/kaggle/working/results/checkpoint-125000/"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(64101, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64101, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [5]:
model.generation_config
start_token = "<unk>"
end_token = "</s>"

In [6]:
from datasets import load_dataset

dataset = load_dataset("mateiaassAI/MEID_v2", split=['train[:97%]', 'train[97%:100%]'])
dataset

Generating train split: 0 examples [00:00, ? examples/s]

[Dataset({
     features: ['wrong', 'right'],
     num_rows: 1967695
 }),
 Dataset({
     features: ['wrong', 'right'],
     num_rows: 60857
 })]

In [7]:
ds_train = dataset[0]
ds_test = dataset[1]
ds_test

Dataset({
    features: ['wrong', 'right'],
    num_rows: 60857
})

In [8]:
punctuation_marks = ['.', '?', '!', ';', '...']

def filter_sentences(sentences):
    text = sentences['right']
    if any(text.endswith(punc) for punc in punctuation_marks):
      words = text.split()
      if len(words) >= 10:
        return True
    return False

In [9]:
fds_test = ds_test.filter(filter_sentences, batched=False)

Filter:   0%|          | 0/60857 [00:00<?, ? examples/s]

In [10]:
fds_test[0]
print(len(fds_test))

31457


In [ ]:
import json

def extract_text(generated_text, start_token="<unk>", end_token="</s>"):
    start_index = generated_text.find(start_token)
    end_index = generated_text.find(end_token, start_index)
    if start_index != -1 and end_index != -1:
        return generated_text[start_index + len(start_token):end_index].strip()
    else:
        return None

sentence_pairs = []
batch_index = 0

for idx, data in enumerate(fds_test):
    print(idx)
    correct_sentence = data['right']
    input_sentence = data['wrong']

    input_tensor = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
    outputs = model.generate(input_tensor)
    generated_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=False)
    predicted_sentence = extract_text(generated_text)

    if predicted_sentence is not None:
        sentence_pairs.append(
            (correct_sentence, input_sentence, predicted_sentence))

    if len(sentence_pairs) >= 1000:
        df = pd.DataFrame(sentence_pairs)
        file_name = f'ress/sentence_pairs_batch_{batch_index}.jsonl'
        with open(file_name, 'w', encoding='utf-8') as f:
            for pair in sentence_pairs:
                json.dump(pair, f, ensure_ascii=False)
                f.write('\n')
        print(f"Saved 1000 sentence pairs to {file_name}")
        sentence_pairs = []  # Clear the list after saving
        batch_index += 1  # Increment the batch index

    if batch_index == 5:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
print(sentence_pairs[0][2])

Pentru perioada în care un pacient are deschisă o fișă de spitalizare de zi pe<unk> parcursul unei singure zile sau pe<unk> parcursul mai multor zile, acesta poate beneficia și de servicii medicale în ambulatoriul de specialitate, altele decât cele necesare acordării serviciilor medicale din spitalizare de zi, cu respectarea condițiilor de acordare a serviciilor medicale în ambulatoriu.


In [26]:
input_ids = tokenizer("Maria sa dus la mare ca sa mannce incchetata cu vanilie ", return_tensors="pt").input_ids
generated_text = tokenizer.decode(model.generate(input_ids)[0], skip_special_tokens=False)

start_index = generated_text.find(start_token)
end_index = generated_text.find(end_token, start_index)

if start_index != -1 and end_index != -1:
    extracted_text = generated_text[start_index + len(start_token):end_index].strip()
else:
    extracted_text = None

print(extracted_text)

Maria sa dus la mare ca sa manance incicheta cu vanilie.
